# COGS 108 - Data Checkpoint

# Names

- Gideon Chan
- Nancy Jiang
- Elliot Kim
- Harrison Nguyen
- Jose Mateo

<a id='research_question'></a>
# Research Question

How well can we model the gas prices of California and the United States? How do the models compare to each other?

# Dataset(s)


- Dataset Name: Weekly U.S. Retail Gasoline Prices
- Link to the dataset:https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=pet&s=emm_epm0_pte_nus_dpg&f=m
- Number of observations: 1561

This dataset contains weekly U.S. retail gasoline prices including all grades and all formulations. It contains 1561 gasoline price data range from 1993-04-05 to 2023-02-27

- Dataset Name: Weekly California Gasoline Prices
- Link to the dataset:https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=pet&s=emm_epm0_pte_sca_dpg&f=w
- Number of observations: 1189

This dataset contains weekly California retail gasoline prices including all grades and all formulations(dollars per gallon). It contains 1189 gasoline price data range from 2000-05-22 to 2023-02-27.

Since the annual and monthly U.S. and California dataset do not have enough data to train the model, so we pick the weekly dataset. We gonna merge those two dataset with exact weekly date, to see how the price shift.

- Dataset Name: All items less food and energy in U.S. city average, all urban consumers, not seasonally adjusted
- Link to the dataset: https://data.bls.gov/timeseries/CUUR0000SA0L1E?output_view=pct_12mths
- Number of observations: 278

This dataset contains the inflation in the U.S., counted monthly from 2000 to 2023. The values types of the inflation is percentage. It contains 278 entires of inflation.


# Setup

In [19]:
import pandas as pd
import numpy as np
import xlrd

# Data Cleaning

Those two datasets are in xlw format, and we need to read each dataset into a dataframe with only the spreadsheet of weekly gasoline prices. Then we match those two dataset with exact same weekly date, so it's fair when we compare the two models after we train them. After we merge the datasets, we extract year and month from the original date, so we can have a better understanding of the data.

In [28]:
df_ca = pd.DataFrame(pd.read_excel("dataset/CA.xls.xlw", 
                                   engine = "xlrd",sheet_name='Data 1'))
df_us = pd.DataFrame(pd.read_excel("dataset/US.xls.xlw",
                                   engine = "xlrd",sheet_name='Data 1'))
df_us_inf = pd.read_csv("dataset/US_Inflation.csv")

In [29]:
df_us.columns=['date','gasoline price']
df_us = df_us.iloc[2:, :].reset_index(drop = True)
df_us 

,date,gasoline price
0,1993-04-05 00:00:00,1.068
1,1993-04-12 00:00:00,1.079
2,1993-04-19 00:00:00,1.079
3,1993-04-26 00:00:00,1.086
4,1993-05-03 00:00:00,1.086
...,...,...
1556,2023-01-30 00:00:00,3.594
1557,2023-02-06 00:00:00,3.552
1558,2023-02-13 00:00:00,3.502
1559,2023-02-20 00:00:00,3.494


In [30]:
df_ca.columns=['date','gasoline price']
df_ca = df_ca.iloc[2:, :].reset_index(drop = True)
df_ca 

,date,gasoline price
0,2000-05-22 00:00:00,1.679
1,2000-05-29 00:00:00,1.673
2,2000-06-05 00:00:00,1.661
3,2000-06-12 00:00:00,1.662
4,2000-06-19 00:00:00,1.664
...,...,...
1184,2023-01-30 00:00:00,4.462
1185,2023-02-06 00:00:00,4.514
1186,2023-02-13 00:00:00,4.535
1187,2023-02-20 00:00:00,4.626


In [31]:
df = pd.merge(df_us, df_ca, on="date")
df.columns = ['date','price_us','price_ca']
df

,date,price_us,price_ca
0,2000-05-22 00:00:00,1.566,1.679
1,2000-05-29 00:00:00,1.579,1.673
2,2000-06-05 00:00:00,1.599,1.661
3,2000-06-12 00:00:00,1.664,1.662
4,2000-06-19 00:00:00,1.711,1.664
...,...,...,...
1184,2023-01-30 00:00:00,3.594,4.462
1185,2023-02-06 00:00:00,3.552,4.514
1186,2023-02-13 00:00:00,3.502,4.535
1187,2023-02-20 00:00:00,3.494,4.626


In [32]:
df['year'] = pd.to_datetime(df['date']).dt.year
df['month'] = pd.to_datetime(df['date']).dt.month

In [33]:
df

,date,price_us,price_ca,year,month
0,2000-05-22 00:00:00,1.566,1.679,2000,5
1,2000-05-29 00:00:00,1.579,1.673,2000,5
2,2000-06-05 00:00:00,1.599,1.661,2000,6
3,2000-06-12 00:00:00,1.664,1.662,2000,6
4,2000-06-19 00:00:00,1.711,1.664,2000,6
...,...,...,...,...,...
1184,2023-01-30 00:00:00,3.594,4.462,2023,1
1185,2023-02-06 00:00:00,3.552,4.514,2023,2
1186,2023-02-13 00:00:00,3.502,4.535,2023,2
1187,2023-02-20 00:00:00,3.494,4.626,2023,2


In [34]:
df_us_inf = df_us_inf.drop("Series ID",axis = 1)
df_us_inf = df_us_inf.drop(df_us_inf[(df_us_inf["Period"] == "S01") | (df_us_inf["Period"] == "S02")].index)
df_us_inf

,Year,Period,Value
0,2000,M01,2.0
1,2000,M02,2.2
2,2000,M03,2.4
3,2000,M04,2.3
4,2000,M05,2.4
...,...,...,...
317,2022,M10,6.3
318,2022,M11,6.0
319,2022,M12,5.7
322,2023,M01,5.6


In [36]:
df_us_inf.loc[df_us_inf["Period"] == "M01", "Period"] = 1
df_us_inf.loc[df_us_inf["Period"] == "M02", "Period"] = 2
df_us_inf.loc[df_us_inf["Period"] == "M03", "Period"] = 3
df_us_inf.loc[df_us_inf["Period"] == "M04", "Period"] = 4
df_us_inf.loc[df_us_inf["Period"] == "M05", "Period"] = 5
df_us_inf.loc[df_us_inf["Period"] == "M06", "Period"] = 6
df_us_inf.loc[df_us_inf["Period"] == "M07", "Period"] = 7
df_us_inf.loc[df_us_inf["Period"] == "M08", "Period"] = 8
df_us_inf.loc[df_us_inf["Period"] == "M09", "Period"] = 9
df_us_inf.loc[df_us_inf["Period"] == "M10", "Period"] = 10
df_us_inf.loc[df_us_inf["Period"] == "M11", "Period"] = 11
df_us_inf.loc[df_us_inf["Period"] == "M12", "Period"] = 12
df_us_inf.columns = ["year","month","inflation"]
df_us_inf

,year,month,inflation
0,2000,1,2.0
1,2000,2,2.2
2,2000,3,2.4
3,2000,4,2.3
4,2000,5,2.4
...,...,...,...
317,2022,10,6.3
318,2022,11,6.0
319,2022,12,5.7
322,2023,1,5.6


In [37]:
df["inflation"] = 0
for row in df.itertuples():
    temp_year = row.year
    temp_month = row.month
    temp_inf = df_us_inf.loc[(df_us_inf["year"] == temp_year) & (df_us_inf["month"] == temp_month)]["inflation"]
    df["inflation"].at[row.Index] = temp_inf
df

,date,price_us,price_ca,year,month,inflation
0,2000-05-22 00:00:00,1.566,1.679,2000,5,2.4
1,2000-05-29 00:00:00,1.579,1.673,2000,5,2.4
2,2000-06-05 00:00:00,1.599,1.661,2000,6,2.5
3,2000-06-12 00:00:00,1.664,1.662,2000,6,2.5
4,2000-06-19 00:00:00,1.711,1.664,2000,6,2.5
...,...,...,...,...,...,...
1184,2023-01-30 00:00:00,3.594,4.462,2023,1,5.6
1185,2023-02-06 00:00:00,3.552,4.514,2023,2,5.5
1186,2023-02-13 00:00:00,3.502,4.535,2023,2,5.5
1187,2023-02-20 00:00:00,3.494,4.626,2023,2,5.5
